In [ ]:
# REMOVE CREDENTIALS BEFORE CHECK-IN TO GIT

# TEST AWS-ACCOUNT 
AWS_ACCESS_KEY_ID = "XXXXXX"
AWS_SECRET_ACCESS_KEY = "XXXXXX"
BUCKET = "XXXXXX"

In [ ]:
import json

import boto3
import pandas as pd

# Credentials from above

s3 = boto3.resource(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

bucket = BUCKET

In [ ]:
keys = []

s3bucket = s3.Bucket(bucket)

for file in s3bucket.objects.all():

    if "ArticlesDev" in file.key:
        keys.append([file.key, str(file.last_modified)])

keys

In [ ]:
df = pd.DataFrame()

for i, key in enumerate(keys):

    if "articles/2019/01/" in key[0]:

        try:
            print("{} : {}".format(i, key[0]))
            obj = s3.Object(bucket, key[0])

            text = obj.get()["Body"].read().decode("utf-8")

            text = "[" + text.replace("}{", "},{") + "]"

            res = json.loads(text)

            df_json = pd.DataFrame(res)
            df_json = df_json.drop(df_json.index[[0]])

            df = df.append(df_json)

        except ValueError:

            print("JSON decoding failed")

In [ ]:
df.head()